# Manga Colorization

Black&White manga/comic colorization, based on pix2pix.

Prerequests: Python 3, GPU # This notebook was tested in datahub

In [1]:
! git clone https://github.com/glh3025/ml-art-project4.git

Cloning into 'ml-art-project4'...
remote: Enumerating objects: 274, done.
remote: Counting objects: 100% (274/274), done.
remote: Compressing objects: 100% (262/262), done.
remote: Total 274 (delta 10), reused 268 (delta 7), pack-reused 0
Receiving objects: 100% (274/274), 37.50 MiB | 33.83 MiB/s, done.
Resolving deltas: 100% (10/10), done.
Checking out files: 100% (243/243), done.


In [2]:
cd ml-art-project4

/datasets/home/home-03/75/075/lgong/ml-art-project4


In [3]:
! pip install -r requirements.txt

The directory '/home/jovyan/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/jovyan/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.


In [4]:
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

--2019-05-30 23:08:41--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving web.ucsd.edu (web.ucsd.edu)... 132.239.1.231, 132.239.1.230
Connecting to web.ucsd.edu (web.ucsd.edu)|132.239.1.231|:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 16648024 (16M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  15.88M  13.8MB/s    in 1.1s    

2019-05-30 23:08:42 (13.8 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [16648024/16648024]



In [5]:
! unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [6]:
# Let Visdom run background, 8097 is the default output port of pix2pix model.
get_ipython().system_raw('python3 -m visdom.server -port 8097 >> visdomlog.txt 2>&1 &')

In [7]:
get_ipython().system_raw('./ngrok http 8097 &')

In [8]:
# Click the link below to access Visdom.
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://af7e1d8d.ngrok.io


In [9]:
# Make B&W images from colored ones to save some time

import os
import numpy as np
import cv2
import argparse

data_dir_A = "datasets/onepiece/A/"
data_dir_B = "datasets/onepiece/B/"
data_dir_AB = "datasets/onepiece_AB/"

splits = os.listdir(data_dir_A)

for sp in splits:
    img_fold_A = os.path.join(data_dir_A, sp)
    img_fold_B = os.path.join(data_dir_B, sp)
    if not os.path.isdir(img_fold_B):
        os.makedirs(img_fold_B)
    img_list = os.listdir(img_fold_A)
    num_imgs = len(img_list)
    print('split = %s, use %d/%d images' % (sp, num_imgs, len(img_list)))
    img_fold_AB = os.path.join(data_dir_AB, sp)
    if not os.path.isdir(img_fold_AB):
        os.makedirs(img_fold_AB)
    print('split = %s, number of images = %d' % (sp, num_imgs))
    for n in range(num_imgs):
        name_A = img_list[n]
        path_A = os.path.join(img_fold_A, name_A)
        name_B = name_A
        path_B = os.path.join(img_fold_B, name_B)
        
        if os.path.isfile(path_A):# and os.path.isfile(path_B):
            name_AB = name_A
            path_AB = os.path.join(img_fold_AB, name_AB)
            im_A = cv2.imread(path_A, 1) # python2: cv2.CV_LOAD_IMAGE_COLOR; python3: cv2.IMREAD_COLOR
#             im_B = cv2.imread(path_B, 1) # python2: cv2.CV_LOAD_IMAGE_COLOR; python3: cv2.IMREAD_COLOR
            im_B = cv2.cvtColor(im_A, cv2.COLOR_BGR2GRAY)
            cv2.imwrite(path_B, im_B)
#             im_AB = np.concatenate([im_A, im_B], 1)
#             cv2.imwrite(path_AB, im_AB)


split = val, use 28/28 images
split = val, number of images = 28
split = test, use 8/8 images
split = test, number of images = 8
split = train, use 141/141 images
split = train, number of images = 141


In [19]:
# If use my pretrained model, download from
https://drive.google.com/file/d/1BcPpAf7FpQ9hO6LiDeYd086UQ3cybden/view?usp=sharing
# and set path ./checkpoints/color_pix2pix/latest_net_G.pth

/bin/sh: 1: w3m: not found


In [10]:
# Train model using ~150 onepiece pages. More pages will be better.
# modified generator model to Unet512 to get higher resolution image.
# By default will train 200 epochs, but ~20 epochs is enough for me.
! python train.py --dataroot ./datasets/onepiece/A/ --name color_pix2pix --model colorization --netG unet_512 --load_size 568 --crop_size 512

----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                crop_size: 512                           	[default: 256]
                 dataroot: ./datasets/onepiece/A/        	[default: None]
             dataset_mode: colorization                  
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: 1                             
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
                    epoch: latest                        

saving the model at the end of epoch 15, iters 2655
End of epoch 15 / 200 	 Time Taken: 41 sec
learning rate = 0.0002000
(epoch: 16, iters: 45, time: 0.268, data: 0.001) G_GAN: 1.204 G_L1: 7.355 D_real: 0.720 D_fake: 0.603 
(epoch: 16, iters: 145, time: 1.730, data: 0.079) G_GAN: 1.613 G_L1: 5.572 D_real: 0.329 D_fake: 0.310 
End of epoch 16 / 200 	 Time Taken: 40 sec
learning rate = 0.0002000
(epoch: 17, iters: 68, time: 0.193, data: 0.001) G_GAN: 1.202 G_L1: 6.875 D_real: 0.391 D_fake: 0.506 
(epoch: 17, iters: 168, time: 0.270, data: 0.001) G_GAN: 0.924 G_L1: 4.429 D_real: 0.572 D_fake: 0.469 
End of epoch 17 / 200 	 Time Taken: 40 sec
learning rate = 0.0002000
(epoch: 18, iters: 91, time: 0.273, data: 0.102) G_GAN: 1.390 G_L1: 7.051 D_real: 1.207 D_fake: 0.188 
End of epoch 18 / 200 	 Time Taken: 39 sec
learning rate = 0.0002000
/opt/conda/lib/python3.6/site-packages/skimage/color/colorconv.py:993: UserWarning: Color data out of range: Z < 0 in 312 pixels
  warn('Color data out of 

In [62]:
# Generate colored pages from B&W testset.
! python test.py --dataroot ./datasets/onepiece/A/ --name color_pix2pix --model colorization --netG unet_512 --load_size 512 --crop_size 512 --num_test 8

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 512                           	[default: 256]
                 dataroot: ./datasets/onepiece/A/        	[default: None]
             dataset_mode: colorization                  
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 1                             
                  isTrain: False                         	[default: None]
                load_iter: 0             

In [63]:
# resize outputs to appropriate resolution

result_dir = "results/color_pix2pix/test_latest/images/"

img_list = os.listdir(result_dir)
num_imgs = len(img_list)
for n in range(num_imgs):
    name_s = img_list[n]
    path_s = os.path.join(result_dir, name_s)
    im_s = cv2.imread(path_s, 1)
    im_l = cv2.resize(im_s, (520,800), interpolation = cv2.INTER_CUBIC)
    cv2.imwrite(path_s, im_l)
    
# The result is in html form stored in results/color_pix2pix/test_latest/